# ChromProcess Introduction Part 2

In ChromProcess Introduction Part 1, peak tables were created from chromatogram files. In Part 2, these peak tables are used to compile series data (e.g. changes in compound concentrations or peak integrals over time).

In [1]:
import os
from ChromProcess import Classes

In [2]:
peak_collection_directory = 'Examples/Example/ExperimentalData/ExamplePeakCollections'

peak_tables = []
for file in os.listdir(peak_collection_directory):
    peak_tables.append(Classes.PeakCollection(file = f'{peak_collection_directory}/{file}'))

In [3]:
conditions_file = 'Examples/Example/ExperimentalData/example_conditions.csv'
conditions = Classes.Experiment_Conditions(information_file = conditions_file)

In [4]:
# Create series of peak collections
series = Classes.PeakCollectionSeries(
                                    peak_tables, 
                                    name = 'Example',
                                    conditions = conditions.conditions
                                    )

Now, there are a bunch of operations which can be performed on this series of peak collections